# View 5D beam density data interactively

Data has been thresholded and interpolated onto a regular grid. The resolution is comparable to the measurement resolution, but the camera resolution (dimensions y3 and x3) have been reduced by a factor of 2.

Coordinates are still in measurement/lab coordinates (slit positions and camera pixels). They have not yet been transformed to phase space coordinates. 
A component of the linear correlation between x1 and x2 has also been removed. The plot axes and slider bar units are in # of grid cells.

$x_3$ = x position at VS06 \
$y_3$ = y position at VS06 \
$x_2$ = position of VT06 slit \
$y_1$ = position of HZ04 slit \
$x_1$ = position of VT04 slit

$x = x_1$ \
$y = y_1$ \
$x' = (x_2- 0.35 x_1)/(s_2-s_1)$ \
$y' ~= (y_3-y_1)/(s_3-s_1)$ \
$w  = f(x_3,x_2,x_1) $ 



In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from ipywidgets import interact

filename = 'data/processed_rawgrid_merged-211203151919-211203192701-scan-xxpy-image-ypdE.npz'
arrays_3d = np.load(filename)['a3d']

In [3]:
arrays_3d_max = arrays_3d.max()
ix1,ix2,iy1,iy3,ix3 = np.where(arrays_3d==arrays_3d_max)
ix1 = ix1[0];  ix2 = ix2[0]; ix3 = ix3[0]; iy1 = iy1[0]; iy3 = iy3[0]
cind = (ix1,ix2,iy1,iy3,ix3)
a3dshape = np.shape(arrays_3d)
print('Center at index (x1,x2,y1,y3,x3) = (%i,%i,%i,%i,%i)'%(ix1,ix2,iy1,iy3,ix3))

arrays_3d = arrays_3d / arrays_3d_max
print('peak value %.2f, normalized to 1'%(arrays_3d_max))

Center at index (x1,x2,y1,y3,x3) = (16,16,15,27,57)
peak value 204814.33, normalized to 1


In [4]:
## Define functions for generating plots

dims = ('x1','x2','y1','y3','x3')

def plot_image(image,log=False):
    nx = np.shape(image)[1]
    ny = np.shape(image)[0]
    xpx = np.arange(nx)
    ypx = np.arange(ny)
    if log:
        floor = np.min(image[image>0])
        plt.pcolor(xpx,ypx,np.log10(floor+image),shading='auto')
        #plt.plot(xpx,0.1*ny*(3+np.log10(profx/profx.max())),'r')
        #plt.plot(0.1*nx*(3+np.log10(profy/profy.max())),ypx,'r')
        #plt.ylim([0,ny])
        #plt.xlim([0,nx])
    else:    
        plt.pcolor(xpx,ypx,image,shading='auto')
        sumima = image.sum().sum()
        if sumima > 0:
            profx = image.sum(axis=0) / sumima
            profy = image.sum(axis=1) / sumima
            plt.plot(xpx,ny*0.2*profx/profx.max(),'w')
            plt.plot(1+nx*0.2*profy/profy.max(),ypx,'w')

def make_slice(dim_names,slices):
    """
    dim_ind is two-element list, indices indicate 
    which axes will be plotted. Other axes will 
    be slider bars.
    """
    #print(slices); print(dim_names)
    slicer = []; 
    counter = 0
    for i in range(len(dims)):
        #print(counter)
        if dims[i] in dim_names:
            slicer.append(slice(0,a3dshape[i]))
        else:
            slicer.append(slices[counter])
            counter += 1
    return slicer


def update_slice(i,j,k,dim1='y3',dim2='x3',cfix=False,clim=arrays_3d_max,log=False):
    dim_names = [dim1,dim2]
    slicer = make_slice(dim_names,[i,j,k])
    ima = arrays_3d[tuple(slicer)]
    plt.figure(1)
    plot_image(ima,log=log); 
    plt.colorbar(); 
    # -- labels 
    # ensure assigned to correct axes
    axes_labels = []
    for dim in dims:
        if dim in dim_names:
            axes_labels.append(dim)
    plt.ylabel(axes_labels[0])
    plt.xlabel(axes_labels[1])
    # -- color axis range
    if cfix:
        if log:
            plt.clim([-1,np.log10(clim)])
        else:
            plt.clim([0,clim])
    plt.show();
    
def make_sum(dim_names,thisdims):
    """
    dim_ind is two-element list, indices indicate 
    which axes will be plotted. 
    Other axes will be summed.
    """
    axes_to_sum = []
    counter = 0
    for i in range(len(thisdims)):
        if thisdims[i] in dim_names:
            counter+=1
        else:
            axes_to_sum.append(counter)
    return axes_to_sum

def update_projection(dim1='y3',dim2='x3',cfix=False,clim=arrays_3d_max,log=False):
    dim_names = [dim1,dim2]
    axes_to_sum = make_sum(dim_names,dims)
    ima = arrays_3d.sum(axes_to_sum[0]).sum(axes_to_sum[1]).sum(axes_to_sum[2])
    plt.figure(1)
    plot_image(ima,log=log); 
    plt.colorbar(); 
    # -- labels 
    # ensure assigned to correct axes
    axes_labels = []
    for dim in dims:
        if dim in dim_names:
            axes_labels.append(dim)
    plt.ylabel(axes_labels[0])
    plt.xlabel(axes_labels[1])
    # -- color axis range
    if cfix:
        if log:
            plt.clim([-1,np.log10(clim)])
        else:
            plt.clim([0,clim])
    plt.show();
    
def update_slice2_proj1(i,j,dim1='x1',dim2='y1',int_dim='x3',cfix=False,clim=1,log=False):
    dim_names = [dim1,dim2,int_dim]
    # -- first select slices in 2 axes
    slicer = make_slice(dim_names,[i,j])
    slice_dims = list(dims)
    [slice_dims.pop(slice_dims.index(dim_names[ii])) for ii in range(len(dim_names))]
    ima_tmp = arrays_3d[tuple(slicer)]

    # -- next sum over 1 axis
    dim_names_2 = [dim1,dim2]
    dim_names_3 = []
    for dim in dims:
        if dim in dim_names:
            dim_names_3.append(dim)
    axes_to_sum = make_sum(dim_names_2,dim_names_3)
    ima = ima_tmp.sum(axes_to_sum[0])


    plt.figure()
    plot_image(ima,log=log); 
    plt.colorbar(); 
    # -- labels 
    # ensure assigned to correct axes
    axes_labels = []
    for dim in dims:
        if dim in dim_names_2:
            axes_labels.append(dim)
    plt.ylabel(axes_labels[0])
    plt.xlabel(axes_labels[1])
    # -- color axis range
    if cfix:
        if log:
            plt.clim([-1,np.log10(clim)])
        else:
            plt.clim([0,clim])
    plt.show();
    
def update_slice1_proj2(i,dim1='x1',dim2='y1',int_dim1='x2',int_dim2='y3',cfix=False,clim=1,log=False):
    dim_names = [dim1,dim2,int_dim1,int_dim2]
    # -- first select slices in 2 axes
    slicer = make_slice(dim_names,[i])
    slice_dims = list(dims)
    [slice_dims.pop(slice_dims.index(dim_names[ii])) for ii in range(len(dim_names))]
    ima_tmp = arrays_3d[tuple(slicer)]

    # -- next sum over 2 axes
    dim_names_2 = [dim1,dim2]
    dim_names_3 = []
    for dim in dims:
        if dim in dim_names:
            dim_names_3.append(dim)
    axes_to_sum = make_sum(dim_names_2,dim_names_3)
    ima = ima_tmp.sum(axes_to_sum[0]).sum(axes_to_sum[1])


    plt.figure()
    plot_image(ima,log=log); 
    plt.colorbar(); 
    # -- labels 
    # ensure assigned to correct axes
    axes_labels = []
    for dim in dims:
        if dim in dim_names_2:
            axes_labels.append(dim)
    plt.ylabel(axes_labels[0])
    plt.xlabel(axes_labels[1])
    # -- color axis range
    if cfix:
        if log:
            plt.clim([-1,np.log10(clim)])
        else:
            plt.clim([0,clim])
    plt.show();
    
    
# -- get slider range info
sliders = []
for i in range(len(dims)):
    sliders.append((0,a3dshape[i],1))

## Full projection

In [5]:
# Create figure
fig= go.FigureWidget()
heatmap = fig.add_heatmap()
fig


@interact(dim1=list(dims),dim2=list(dims),logscale=False)
def update(dim1='y3',dim2='x3',logscale=False):
    with fig.batch_update():
        ima = update_projection(dim1=dim1,dim2=dim2,log=logscale)
        heatmap.data = ima


interactive(children=(Dropdown(description='dim1', index=3, options=('x1', 'x2', 'y1', 'y3', 'x3'), value='y3'…

## View 2D slice in 5D density

dim1 and dim2 are the names of the dimensions to plot. Move the slider bars to select which slice you are viewing. Note that the slider bars for dim1 and dim2 are not be active.

In [6]:
# Create figure
fig = go.FigureWidget()
heatmap = fig.add_heatmap()

        
@interact(dim1=list(dims),dim2=list(dims),
          x1 = sliders[0], x2 = sliders[1], y1 = sliders[2], y3 = sliders[3], x3 = sliders[4],
          clim_fix=False,clim=(0,1,.01),logscale=False)
def update(dim1='y3',dim2='x3',
           x1=cind[0],x2=cind[1],y1=cind[2],y3=cind[3],x3=cind[4],
           logscale=False,clim_fix=False,clim=arrays_3d_max):
    
    slices = []
    for i in range(len(dims)):
        if not(dims[i] in [dim1,dim2]):
            slices.append(eval(dims[i]))
            
    with fig.batch_update():
        ima = update_slice(slices[0],slices[1],slices[2],dim1=dim1,dim2=dim2,cfix=clim_fix,clim=clim,log=logscale)
        heatmap.data = ima



interactive(children=(Dropdown(description='dim1', index=3, options=('x1', 'x2', 'y1', 'y3', 'x3'), value='y3'…

## View 2D slice in 4D density

dim1 and dim2 are the names of the dimensions to plot. int_dim is the name of the dimension to project along. 

Move the slider bars to select which slice you are viewing. Note that the slider bars for dim1, dim2 and int_dim are not be active.

In [7]:
# Create figure
fig = go.FigureWidget()
heatmap = fig.add_heatmap()

        
@interact(dim1=list(dims),dim2=list(dims),int_dim= list(dims),
          x1 = sliders[0], x2 = sliders[1], y1 = sliders[2], y3 = sliders[3], x3 = sliders[4],
          clim_fix=False,clim=(0,20,.01),logscale=False)
def update(dim1='y3',dim2='x3',int_dim='x1',
           x1=cind[0],x2=cind[1],y1=cind[2],y3=cind[3],x3=cind[4],
           logscale=False,clim_fix=False,clim=10):
    
    slices = []
    for i in range(len(dims)):
        if not(dims[i] in [dim1,dim2,int_dim]):
            slices.append(eval(dims[i]))
            
    with fig.batch_update():
        ima = update_slice2_proj1(slices[0],slices[1],dim1=dim1,dim2=dim2,int_dim=int_dim,cfix=clim_fix,clim=clim,log=logscale)
        heatmap.data = ima



interactive(children=(Dropdown(description='dim1', index=3, options=('x1', 'x2', 'y1', 'y3', 'x3'), value='y3'…

## View 2D slice in 3D density

dim1 and dim2 are the names of the dimensions to plot. int_dim1 and int_dim2 are the names of the dimensions to project along. 

Move the slider bars to select which slice you are viewing. Note that the slider bars for dim1, dim2, int_dim1 and int_dim2 are not active.

In [8]:
# Create figure
fig = go.FigureWidget()
heatmap = fig.add_heatmap()
fig

        
@interact(dim1=list(dims),dim2=list(dims),int_dim1= list(dims),int_dim2= list(dims),
          x1 = sliders[0], x2 = sliders[1], y1 = sliders[2], y3 = sliders[3], x3 = sliders[4],
          clim_fix=False,clim=(0,100,.01),logscale=False)
def update(dim1='x1',dim2='y1',int_dim1='x2',int_dim2='y3',
           x1=cind[0],x2=cind[1],y1=cind[2],y3=cind[3],x3=cind[4],
           logscale=False,clim_fix=False,clim=10):
    
    slices = []
    for i in range(len(dims)):
        if not(dims[i] in [dim1,dim2,int_dim1,int_dim2]):
            slices.append(eval(dims[i]))
            
    with fig.batch_update():
        ima = update_slice1_proj2(slices[0],dim1=dim1,dim2=dim2,int_dim1=int_dim1,int_dim2=int_dim2,cfix=clim_fix,clim=clim,log=logscale)
        heatmap.data = ima


interactive(children=(Dropdown(description='dim1', options=('x1', 'x2', 'y1', 'y3', 'x3'), value='x1'), Dropdo…